In [3]:
import httpx
from fasthtml.common import show
from bs4 import BeautifulSoup
from fasthtml.common import L, first, last
from fastcore.xtras import globtastic
from fastcore.parallel import parallel
import os
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
# I just copied this from devTools -> dashboard request -> copy as cURL. You will have to replace the session_ cookie with your own.

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8,ca;q=0.7,zh-TW;q=0.6,zh;q=0.5',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"'
}

r = httpx.get('https://solveit.fast.ai/dashboard', 
              headers=headers,
              cookies={'session_': os.getenv('SESSION_COOKIE')},
              follow_redirects=True)


In [5]:
soup = BeautifulSoup(r.text, 'html.parser')


In [6]:
links = L(soup.select('ul.uk-list li a[href]')).map(lambda x: x['href']).filter(lambda x:'youtu' in x)
links

(#12) ['https://youtu.be/4x9wtrDOXac','https://youtube.com/live/X_m7qz5LteY?feature=share','https://youtube.com/live/kEGQSamwpuU?feature=share','https://youtube.com/live/HO1gmouGEVI?feature=share','https://youtube.com/live/w_YltetZc3M?feature=share','https://youtube.com/live/83tbt5lEUgI?feature=share','https://youtube.com/live/Dnu39sQCV3k?feature=share','https://youtube.com/live/lA-KvHeFJkI?feature=share','https://youtube.com/live/ObliDyAdRx8?feature=share','https://youtube.com/live/5ZPFC_Xnz8A?feature=share','https://youtube.com/live/AvItkLCD7v8?feature=share','https://youtube.com/live/rr-59PLkIdY?feature=share']

## Download videos


In [7]:
import yt_dlp

def download_video(url, output_path='%(title)s.%(ext)s'):
    ydl_opts = {
        'format': 'best',  # best quality
        'outtmpl': output_path,  # output template
        'quiet': False,  # show progress
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [8]:

# Download a single video
# download_video('https://youtu.be/4x9wtrDOXac')


In [9]:
def download_audio(url, output_path='./audio/%(title)s.%(ext)s'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
        'quiet': False,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [10]:
# download_audio('https://youtu.be/4x9wtrDOXac')


In [11]:
from fastcore.parallel import parallel

# parallel(download_audio, links)

## Diarize audio


Unfortunately, HTTP GET method has a payload size limit, so providing the file direclty is not possible. It's possible to work around uploading it somewhere which is publicly accessible and then use a URL. Instead, we'll manually use macWhisper for now.

[See issue](https://github.com/replicate/replicate-python/issues/157)

In [12]:
import replicate

def diarize_with_replicate(filepath, num_speakers=2, language="en", prompt=""):
    input = {
        "file": open(filepath, "rb"),
        "prompt": prompt,
        "language": language,
        "num_speakers": num_speakers
    }
    
    output = replicate.run(
        "thomasmol/whisper-diarization:cbd15da9f839c5f932742f86ce7def3a03c22e2b4171d42823e83e314547003f",
        input=input
    )
    return output

# Run diarization
# result = diarize_with_replicate(
#     "./audio/SIWC - lesson1.mp3", 
#     prompt="SolveIT lesson, coding tutorial",
#     num_speakers=2
# )
# print(result)

ReplicateError: ReplicateError Details:
status: 413

In [12]:
# this will also fail as it uses Replicate behind the scenes
import httpx

def diarize_audio(filepath, instruction="I want to diarize this audio file of a SolveIT lesson in English with Jeremy Howard and Johnathan Whitaker speaking."):
    url = "https://web-production-1ac0.up.railway.app/diarize/chat"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {TOKEN}"
    }
    
    files = {
        "file": ("audio.mp3", open(filepath, "rb")),
        "instruction": (None, instruction)
    }
    
    r = httpx.post(url, headers=headers, files=files)
    return r.json()

# Assuming the first downloaded file is named by its title
# Get the first downloaded file path
# first_file = "SIWC - lesson1.mp3"  # adjust this to your actual filename
# result = diarize_audio(first_file)
# print(result)

## Produce titles

In [13]:
t1 = open('./transcripts/SIWC - lesson1.txt').read()
t1[:100]

'Before we kind of have our official start, I just wanted to do a couple of quick logistics things, w'

In [14]:
from claudette import Chat, Client, models
model = models[1]
model

'claude-3-5-sonnet-20241022'

In [15]:
chat = Chat(model, sp="You are a helpful assistant that produces titles, and summaries for SolveIT lessons.")
chat(t1)

Here's a title and summary for this lesson:

Title: "Introduction to Dialogue Engineering with Python and Advent of Code"

Summary:
This first lesson introduces the SolveIT platform and dialogue engineering approach to problem solving with AI and code. The instructors cover:

1. Platform logistics and usage:
- Overview of SolveIT features and interface
- Discord community setup and best practices
- Resource quotas and limitations

2. Core concepts:
- Dialogue engineering as an iterative approach combining code and AI assistance
- Exploratory programming methodology
- Importance of checking data and documentation at each step

3. Practical demonstration:
- Solving Advent of Code 2023 Day 1A problem
- Breaking down problems into small, testable steps
- Using Python basics (strings, lists, loops) with AI assistance
- Testing with sample data before scaling to full input

4. Best practices:
- Writing clear, testable code
- Documenting and organizing work in dialogues
- Learning from AI suggestions by reading documentation
- Maintaining control of the development process

The lesson concludes with homework to complete Day 1B and optionally Day 2, encouraging students to practice the demonstrated techniques.

<details>

- id: `msg_01MjjcLB9KqoK7C1NCPAWCrx`
- content: `[{'citations': None, 'text': 'Here\'s a title and summary for this lesson:\n\nTitle: "Introduction to Dialogue Engineering with Python and Advent of Code"\n\nSummary:\nThis first lesson introduces the SolveIT platform and dialogue engineering approach to problem solving with AI and code. The instructors cover:\n\n1. Platform logistics and usage:\n- Overview of SolveIT features and interface\n- Discord community setup and best practices\n- Resource quotas and limitations\n\n2. Core concepts:\n- Dialogue engineering as an iterative approach combining code and AI assistance\n- Exploratory programming methodology\n- Importance of checking data and documentation at each step\n\n3. Practical demonstration:\n- Solving Advent of Code 2023 Day 1A problem\n- Breaking down problems into small, testable steps\n- Using Python basics (strings, lists, loops) with AI assistance\n- Testing with sample data before scaling to full input\n\n4. Best practices:\n- Writing clear, testable code\n- Documenting and organizing work in dialogues\n- Learning from AI suggestions by reading documentation\n- Maintaining control of the development process\n\nThe lesson concludes with homework to complete Day 1B and optionally Day 2, encouraging students to practice the demonstrated techniques.', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 19032, 'output_tokens': 267}`

</details>

In [16]:
from fastcore.basics import BasicRepr
class Summary(BasicRepr):
    "Information about a SolveIT lesson"
    def __init__(self, 
                title:str, # title of the lesson
                summary:str, # summary of the lesson
                youtube_description:str, # youtube description of the lesson
                ):
        self.title = title
        self.summary = summary
        self.youtube_description = youtube_description

cli = Client(model)
s = cli.structured(f"You are a helpful assistant that produces titles, and summaries for SolveIT lessons. Provide information from the following SolveIT lesson transcript. {t1}", Summary)
s


[Summary(title='SolveIT Platform Introduction and First Coding Problem', summary='This lesson introduces the SolveIT platform and walks through solving the first Advent of Code 2023 problem. Key points covered:\n\n1. Platform logistics and usage guidelines:\n- 2GB disk quota limit\n- Being mindful of CPU usage\n- Discord navigation and organization tips\n\n2. Introduction to SolveIT platform features:\n- Code, notes, and AI assistance integration\n- Variable tracking and autocompletion\n- Dialogue-based problem solving approach\n\n3. Practical coding demonstration:\n- Solving Advent of Code Day 1 Part A\n- Step-by-step problem breakdown\n- Using Python basics with string manipulation\n- Testing and iterative development approach\n\n4. Teaching methodology:\n- Dialogue engineering concept\n- Importance of checking documentation\n- Breaking problems into small testable steps', youtube_description='Learn about the new SolveIT platform and follow along as Jeremy Howard and Jono Lange demon

In [21]:
def summarize_lesson(filepath):
    t = open(filepath).read()
    cli = Client(model)
    return filepath, cli.structured(f"You are a helpful assistant that produces titles, and summaries for SolveIT lessons. Provide information from the following SolveIT lesson transcript. {t}", Summary)


In [22]:
# let's get all the transcripts
ts = globtastic('./transcripts', file_glob='*.txt')
ts


(#12) ['transcripts/SIWC Lesson 8.txt','transcripts/SIWC Lesson 9.txt','transcripts/SIWC - lesson1.txt','transcripts/SIWC Lesson 11.txt','transcripts/SIWC Lesson 10.txt','transcripts/SIWC Lesson 12.txt','transcripts/SIWC Lesson 2.txt','transcripts/SIWC Lesson 3.txt','transcripts/SIWC Lesson 7.txt','transcripts/SIWC Lesson 6.txt','transcripts/SIWC Lesson 4.txt','transcripts/SIWC Lesson 5.txt']

In [19]:
# I had some error running in parallel not sure why
# summaries = parallel(summarize_lesson, ts)

In [20]:
summaries = ts.map(summarize_lesson)
summaries

(#12) [('transcripts/SIWC Lesson 8.txt', [Summary(title='Lean Startup, Iteration, and Building Better Tools with Eric Ries', summary="Eric Ries, co-founder of Ansarai and author of The Lean Startup, discusses the importance of rapid iteration, feedback loops, and building tools that enable faster learning cycles. He shares insights from his experience developing the Lean Startup methodology and how similar principles apply to software development and AI tools. Key themes include the value of small batch sizes, the importance of clearly defined expectations and hypotheses, and creating environments that enable serendipitous innovation. Eric also provides feedback on SolveIT's approach and new features like ghost text completion.", youtube_description="Join Eric Ries, co-founder of Ansarai and author of The Lean Startup, for an insightful discussion about:\n\n- The principles behind Lean Startup methodology and rapid iteration\n- Why small batch sizes and quick feedback loops are crucial

In [23]:
import json
import yaml
from pathlib import Path

# Convert summaries to a more serializable format
def process_summaries(summaries):
    result = {}
    for filepath, summary_list in summaries:
        # Get just the filename without path and extension
        name = Path(filepath).stem
        # Take first summary from list (since they're all length 1)
        summary = summary_list[0]
        result[name] = {
            'title': summary.title,
            'summary': summary.summary,
            'youtube_description': summary.youtube_description
        }
    return result

processed = process_summaries(summaries)

# Export to YAML
with open('summaries.yaml', 'w') as f:
    yaml.dump(processed, f, sort_keys=False, allow_unicode=True)